In [1]:
!rm -r /kaggle/working/*
%cd /kaggle/working

/kaggle/working


In [2]:
import os
import sys

PACKAGE_DIR = "/kaggle/src"
sys.path.append(PACKAGE_DIR)

In [3]:
import whoosh_utils

Processing /kaggle/input/whoosh-wheel-2-7-4/Whoosh-2.7.4-py2.py3-none-any.whl
Whoosh is already installed with the same version as the provided wheel. Use --force-reinstall to force an installation of the wheel.


In [4]:
import gc
import warnings

import numpy as np
import polars as pl
import yaml
from tqdm import tqdm

warnings.filterwarnings("ignore")

In [5]:
train = pl.read_parquet("/kaggle/input/uspto-train-data/train_1000.parquet")
train.head(1)

publication_number,target_0,target_1,target_2,target_3,target_4,target_5,target_6,target_7,target_8,target_9,target_10,target_11,target_12,target_13,target_14,target_15,target_16,target_17,target_18,target_19,target_20,target_21,target_22,target_23,target_24,target_25,target_26,target_27,target_28,target_29,target_30,target_31,target_32,target_33,target_34,target_35,target_36,target_37,target_38,target_39,target_40,target_41,target_42,target_43,target_44,target_45,target_46,target_47,target_48,target_49
str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str
"""US-5292324-A""","""US-5662659-A""","""US-4959070-A""","""US-4813957-A""","""US-5752960-A""","""US-7186258-B2""","""US-4957505-A""","""US-4785810-A""","""US-5556400-A""","""US-5549614-A""","""US-5176701-A""","""US-4715373-A""","""US-7011666-B2""","""US-9277989-B2""","""US-6214015-B1""","""US-5234436-A""","""US-6206887-B1""","""US-2005143750-…","""US-7947049-B2""","""US-5217464-A""","""US-5207708-A""","""US-5312413-A""","""US-5163419-A""","""US-6811553-B2""","""US-5007913-A""","""US-8888845-B2""","""US-7422604-B2""","""US-5584304-A""","""US-2005149057-…","""US-6113606-A""","""US-4844065-A""","""US-4976716-A""","""US-5571113-A""","""US-5395378-A""","""US-5702400-A""","""US-5267553-A""","""US-2004193263-…","""US-4906247-A""","""US-6267768-B1""","""US-5876440-A""","""US-5066297-A""","""US-5728117-A""","""US-8574293-B2""","""US-4726367-A""","""US-5026393-A""","""US-2021244531-…","""US-6241737-B1""","""US-2009030424-…","""US-4655775-A""","""US-2022370195-…","""US-10744032-B2…"


In [6]:
import json

with open("/kaggle/input/cpc-counts/patent2cpc.json", "r") as f:
    patent2cpc = json.load(f)

In [7]:
with open("/kaggle/input/cpc-mapping/cpc2patents.json", "r") as f:
    cpc2patents = json.load(f)

In [23]:
for i, target_ids in enumerate(train.iter_rows()):
    target_ids = set(list(target_ids)[1:])
    assert len(target_ids) == 50

    all_cpcs = set(sum([patent2cpc[target_id] for target_id in target_ids], []))
    inner_outers = []
    for cpc in all_cpcs:
        inner_set = set()
        outer_set = set()
        for patent in cpc2patents[cpc]:
            if patent in target_ids:
                inner_set.add(patent)
            else:
                outer_set.add(patent)
        inner_outers.append((len(inner_set), len(outer_set)))
    print(sorted(inner_outers, reverse=True)[:10])

    if i == 200:
        break

[(28, 254), (10, 228), (6, 1236), (5, 1332), (5, 909), (5, 190), (4, 2959), (4, 790), (4, 253), (4, 163)]
[(4, 220), (3, 953), (3, 223), (2, 3739), (2, 1785), (2, 568), (2, 316), (2, 264), (2, 224), (1, 6876)]
[(11, 482), (10, 1492), (6, 1423), (6, 961), (6, 431), (5, 371), (5, 283), (4, 216), (3, 1125), (3, 945)]
[(11, 1145), (9, 733), (7, 721), (7, 196), (6, 718), (5, 1213), (5, 570), (4, 582), (4, 202), (3, 1427)]
[(7, 428), (6, 877), (5, 498), (4, 295), (3, 734), (3, 381), (3, 217), (3, 170), (3, 157), (3, 47)]
[(12, 460), (10, 20082), (6, 142), (5, 462), (4, 737), (4, 379), (4, 186), (4, 153), (4, 147), (4, 26)]
[(12, 22631), (6, 14117), (4, 27425), (4, 14125), (4, 11680), (4, 5487), (4, 2701), (3, 16012), (3, 13425), (3, 5539)]
[(17, 2473), (17, 2461), (14, 2111), (12, 3466), (12, 1295), (9, 3948), (9, 2017), (9, 1898), (9, 1727), (7, 3822)]
[(12, 4273), (11, 3200), (11, 2214), (8, 8056), (7, 9178), (7, 598), (6, 1833), (6, 1617), (6, 490), (5, 5078)]
[(6, 3305), (4, 630), (3, 35

In [22]:
cpcs = [patent2cpc[target_id] for target_id in target_ids]
cpcs

[['A61B17/29', 'A61B17/30', 'A61B2217/007', 'A61F2/1664', 'A61M3/0279'],
 ['A61B17/0231', 'A61F2/1662', 'A61F9/007'],
 ['A61B17/320016',
  'A61B2017/32113',
  'A61B2090/08021',
  'A61F2/1662',
  'A61F9/00754',
  'A61F9/0133'],
 ['A61B17/29', 'A61B17/30', 'A61B2217/007', 'A61F2/1664', 'A61M3/0279'],
 ['A61B17/29', 'A61B2017/2837', 'A61F2/1664'],
 ['A61F2/1613', 'A61F2002/1683'],
 ['A61F2/1664', 'Y10S623/907'],
 ['A61F9/00736'],
 ['A61F2/1616', 'A61F2/1664', 'A61F2/1691'],
 ['A61F2/1678', 'A61F2002/30217', 'A61F2230/0067'],
 ['A61F2/1664'],
 ['A61B17/30', 'A61F2/16', 'A61F2/1662', 'A61F2/1664'],
 ['A61F2/1664', 'Y10S623/905'],
 ['A61F2/1664', 'A61F2/1667', 'A61F2/167', 'A61F2/1675'],
 ['A61B17/32053', 'A61F9/00736', 'A61F9/00754'],
 ['A61F2/16', 'A61F2/167', 'A61F2/1678', 'A61F2/1691'],
 ['A61F2/1602', 'A61F2/1613', 'A61F2/1662'],
 ['A61F2/1664'],
 ['A61F2/1664'],
 ['A61F9/00736', 'A61F9/00745'],
 ['A61F2/1678', 'A61F2/1691'],
 ['A61F2/1664'],
 ['A61F9/00754'],
 ['A61F2/1662', 'B25B9/02'